<a href="https://www.kaggle.com/code/fhloston/the-fremen?scriptVersionId=182390150" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
def convert_to_int(value):
    if pd.isnull(value):
        return 0
    value = str(value)
    if value.endswith('B'):
        return int(float(value[:-1]) * 1_000_000_000)
    elif value.endswith('M'):
        return int(float(value[:-1]) * 1_000_000)
    elif value.endswith('K'):
        return int(float(value[:-1]) * 1_000)
    return int(value)

In [2]:
def convert_to_float(value):
    if pd.isnull(value):
        return 0.0
    else:
        return round(float(value.replace('$', '').replace(',', '')), 2)

In [3]:
# Function to format numbers as millions or billions with two decimal places
def format_millions_billions(value):
    if pd.isnull(value):
        return '0'
    if value >= 1_000_000_000:
        return f"{value / 1_000_000_000:.2f}B"
    elif value >= 1_000_000:
        return f"{value / 1_000_000:.2f}M"
    else:
        return f"{value:.2f}"

In [4]:
# Function to format numbers as dollars with two decimal places
def format_dollars(value):
    if pd.isnull(value):
        return '$0'
    else:
        return f'${value:.2f}'

In [5]:
from datetime import datetime

def convert_date(date_str, input_format='%d-%b-%y', output_format='%Y-%m-%d'):
    try:
        # Convert the date string to a datetime object using the input format
        date_obj = datetime.strptime(date_str, input_format)
        # Convert the datetime object back to a string using the output format
        converted_date = date_obj.strftime(output_format)
        return converted_date
    except ValueError:
        return 'Invalid Date Format'

In [6]:
import pandas as pd

import pandasql as psql

file_path = '/kaggle/input/ipo-stock/IPO.csv'

IPO = pd.read_csv(file_path)

IPO.head()

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,WAY,Waystar Holding Corp.,$21.50,$20.70,-3.72%,3.45B,967.50M,NASDAQ
1,7-Jun-24,RAPP,"Rapport Therapeutics, Inc.",$17.00,$20.80,22.35%,735.83M,136.00M,NASDAQ
2,7-Jun-24,CHEB,Chenghe Acquisition II Co.,$10.00,$10.02,0.20%,100.20M,75.00M,NYSEAMERICAN
3,6-Jun-24,GAUZ,Gauzy Ltd.,$17.00,$15.74,-7.41%,295.29M,75.00M,NASDAQ
4,6-Jun-24,FLYE,"Fly-E Group, Inc.",$4.00,$4.00,NaN,100.00M,9.00M,NASDAQ


In [7]:
# Specify the columns that need conversion
columns_to_convert = ['MarketCap', 'DealSize']
columns_to_float_convert = ['IPOPrice', 'Current']

# Convert specified columns to integer
for column in columns_to_convert:
    IPO[column] = IPO[column].apply(convert_to_int)

# Convert specified columns to float
for column in columns_to_float_convert:
    IPO[column] = IPO[column].apply(convert_to_float)

# Print the DataFrame to verify the conversion
IPO.head()

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,WAY,Waystar Holding Corp.,21.5,20.70,-3.72%,3450000000,967500000,NASDAQ
1,7-Jun-24,RAPP,"Rapport Therapeutics, Inc.",17.0,20.80,22.35%,735830000,136000000,NASDAQ
2,7-Jun-24,CHEB,Chenghe Acquisition II Co.,10.0,10.02,0.20%,100200000,75000000,NYSEAMERICAN
3,6-Jun-24,GAUZ,Gauzy Ltd.,17.0,15.74,-7.41%,295290000,75000000,NASDAQ
4,6-Jun-24,FLYE,"Fly-E Group, Inc.",4.0,4.00,NaN,100000000,9000000,NASDAQ


# Dates

In [8]:
start_date = '4-Jan-19'
end_date = '9-Jun-24'

# SPAC Companies

In [9]:
query = f"""
Select *
from IPO
where CompanyName like '%Acquisition%'
and IPODate between '{start_date}' AND '{end_date}'
"""

result = psql.sqldf(query, locals())

result['MarketCap'] = result['MarketCap'].apply(format_millions_billions)
result['DealSize'] = result['DealSize'].apply(format_millions_billions)

result['IPOPrice'] = result['IPOPrice'].apply(format_dollars)
result['Current'] = result['Current'].apply(format_dollars)

display(result)

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,CHEB,Chenghe Acquisition II Co.,$10.00,$10.02,0.20%,100.20M,75.00M,NYSEAMERICAN
1,9-Feb-24,HLXB,Helix Acquisition Corp. II,$10.00,$10.34,3.40%,235.98M,160.00M,NASDAQ
2,6-Oct-23,SPKL,Spark I Acquisition Corp. Class A Ordinary Share,$10.00,$10.34,3.40%,169.80M,100.00M,NASDAQ
3,6-Oct-23,QETA,Quetta Acquisition Corporation,$10.00,$10.34,3.35%,92.47M,60.00M,NASDAQ
4,5-May-23,ALCY,Alchemy Investments Acquisition Corp 1,$10.00,$10.69,6.90%,160.03M,100.00M,NASDAQ
5,4-Oct-22,AQU,Aquaron Acquisition Corp.,$10.00,$10.95,9.50%,49.86M,50.00M,NASDAQ
6,7-Apr-22,DECA,Denali Capital Acquisition Corp.,$10.00,$7.00,-30.00%,49.77M,75.00M,NASDAQ
7,9-Feb-22,BYNO,byNordic Acquisition Corporation,$10.00,$11.27,12.70%,115.81M,150.00M,NASDAQ
8,9-Feb-22,CPAQ,Counter Press Acquisition Corporation,$10.00,$10.33,3.30%,95.54M,75.00M,NASDAQ
9,8-Feb-22,LIVB,LIV Capital Acquisition Corp. II,$10.00,$10.97,9.70%,127.36M,100.00M,NASDAQ


# REIT Companies

In [10]:
query = f"""
Select * 
from IPO 
where CompanyName like '%Trust%' 
--and IPODate between '{start_date}' AND '{end_date}'
"""

result = psql.sqldf(query, locals())

result['MarketCap'] = result['MarketCap'].apply(format_millions_billions)
result['DealSize'] = result['DealSize'].apply(format_millions_billions)

result['IPOPrice'] = result['IPOPrice'].apply(format_dollars)
result['Current'] = result['Current'].apply(format_dollars)

display(result)

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Oct-20,SQFT,"Presidio Property Trust, Inc.",$5.00,$0.82,-83.70%,11.86M,2.50M,NASDAQ
1,22-Nov-19,PINE,"Alpine Income Property Trust, Inc.",$19.00,$15.43,-18.79%,210.21M,0.00,NYSE
2,15-May-19,PSTL,"Postal Realty Trust, Inc.",$17.00,$13.31,-21.71%,370.03M,0.00,NYSE


# MAX IPOPrice

In [11]:
query = f"""
Select Symbol, CompanyName, IPODate, max(IPOPrice)
from IPO
--where IPODate between '{start_date}' AND '{end_date}'
--group by Symbol, CompanyName, IPODate, Exchange

"""

result = psql.sqldf(query, locals())

result['max(IPOPrice)'] = result['max(IPOPrice)'].apply(format_dollars)
#result['min(IPOPrice)'] = result['min(IPOPrice)'].apply(format_dollars)
display(result)

,Symbol,CompanyName,IPODate,max(IPOPrice)
0,SNOW,Snowflake Inc.,16-Sep-20,$120.00


# MAX MarketCap

In [12]:
query = f"""
Select Symbol, CompanyName, IPODate, max(MarketCap) 
from IPO 
--where IPODate between '{start_date}' AND '{end_date}'
--group by Symbol, CompanyName, IPODate, Exchange
"""

result = psql.sqldf(query, locals())

# result['min(MarketCap)'] = result['min(MarketCap)'].apply(format_millions_billions)
result['max(MarketCap)'] = result['max(MarketCap)'].apply(format_millions_billions)

display(result)

,Symbol,CompanyName,IPODate,max(MarketCap)
0,UBER,"Uber Technologies, Inc.",10-May-19,144.82B


# Big Deals


In [13]:
query = f"""
Select Symbol, CompanyName, IPODate, DealSize
from IPO 
where --IPODate between '{start_date}' and '{end_date}'
--and 
DealSize > 100000000
order by DealSize desc
"""

result = psql.sqldf(query, locals())

result['DealSize'] = result['DealSize'].apply(format_millions_billions)

display(result)

,Symbol,CompanyName,IPODate,DealSize
0,ARM,Arm Holdings plc,14-Sep-23,4.87B
1,KVUE,Kenvue Inc.,4-May-23,3.80B
2,ABNB,"Airbnb, Inc.",10-Dec-20,3.51B
3,DASH,"DoorDash, Inc.",9-Dec-20,3.37B
4,SNOW,Snowflake Inc.,16-Sep-20,3.36B
...,...,...,...,...
303,NUBI,Nubia Brand International Corp.,11-Mar-22,110.00M
304,NUBI,Nubia Brand International Corp.,11-Mar-22,110.00M
305,BWAC,Better World Acquisition Corp.,13-Nov-20,110.00M
306,NNOX,Nano-X Imaging Ltd.,21-Aug-20,105.88M


# MAX Big Deals

In [14]:
query = f"""
Select Symbol, CompanyName, IPODate, max(MarketCap) as 'max(BigDeals)' from IPO
where DealSize >=100000000
"""

result = psql.sqldf(query, locals())

# result['MarketCap'] = result['MarketCap'].apply(format_millions_billions)
result['max(BigDeals)'] = result['max(BigDeals)'].apply(format_millions_billions)

display(result)

,Symbol,CompanyName,IPODate,max(BigDeals)
0,ARM,Arm Holdings plc,14-Sep-23,142.08B


# Exchange Statistics

In [15]:
query = f"""
Select Exchange, count(Symbol) as 'Companies went public'
from IPO 
--where IPODate between '{start_date}' and '{end_date}'
"""

result = psql.sqldf(query, locals())
display(result)

,Exchange,Companies went public
0,NASDAQ,1307


# Exchange with biggest IPO count

In [16]:
query = f"""
Select Exchange, count(Symbol) as 'Highest number of IPOs'
from IPO 
--where IPODate between '{start_date}' and '{end_date}'
group by Exchange
order by count(Symbol) desc
Limit 1
"""

result = psql.sqldf(query, locals())
display(result)

,Exchange,Highest number of IPOs
0,NASDAQ,781


# IPO Positive return

In [17]:
query = f"""
SELECT COUNT(Symbol) AS PositiveReturnCount 
FROM IPO 
WHERE Return > 0
"""

result = psql.sqldf(query, locals())
display(result)

,PositiveReturnCount
0,571


In [18]:
# Specify the columns that need conversion
columns_to_date_convert = ['IPODate']


# Convert specified columns to integer
for column in columns_to_date_convert:
    IPO[column] = IPO[column].apply(convert_date)

# Total market capitalization

In [19]:
start_date = '2024-03-09'
end_date = '2024-06-09'

query = f"""
SELECT SUM(MarketCap) AS TotalMarketCap 
FROM IPO
where IPODate between '{start_date}' and '{end_date}'
"""

result = psql.sqldf(query, locals())

result['TotalMarketCap'] = result['TotalMarketCap'].apply(format_millions_billions)

display(result)


,TotalMarketCap
0,130.99B
